## Initialize

In [1]:
from dummy_spark import SparkContext
# from pyspark import SparkContext

In [2]:
from sympy import *
from drudge import *
from agp_fermi import *

In [3]:
ctx = SparkContext()

In [4]:
dr = AGPFermi(ctx)

In [5]:
names = dr.names

In [39]:
p, q , r, s  = names.A_dumms[:4]

## Creation / annihilation operators

In [40]:
c_p = dr.sum(names.c_[p])

In [41]:
cdag_p = dr.sum(names.c_dag[q])

In [42]:
cp2 = names.c_[p]
type(cp2)

drudge.term.Vec

In [43]:
print(cp2.base)
print(cp2.indices)

c
(CranChar.AN, p)


In [44]:
dr.simplify(c_p*cdag_p + cdag_p*c_p).display()

<IPython.core.display.Math object>

In [45]:
(dr.fermi_dr.dagger(c_p)).display()

<IPython.core.display.Math object>

## Pairing operators

In [46]:
P_p = dr.sum( dr.P[p] )
Pdag_q = dr.sum( dr.Pdag[q] )

In [47]:
Pdag_q.display()

<IPython.core.display.Math object>

In [48]:
P_p.display()

<IPython.core.display.Math object>

In [49]:
expr = dr.simplify(P_p * Pdag_q - Pdag_q * P_p)
expr.display()

<IPython.core.display.Math object>

## Spin range sum

In [50]:
dr.sum(names.c_[p,UP]).display()

<IPython.core.display.Math object>

In [51]:
(dr.sum((s,dr.spin_range),names.c_[p,s])).simplify().display()

<IPython.core.display.Math object>

## Spin flip su(2)

In [52]:
Sp = dr.sum(names.J_p[p])
Sm = dr.sum(names.J_m[q])

In [53]:
Sp.display()

<IPython.core.display.Math object>

In [54]:
expr = dr.simplify( Sp * Sm - Sm * Sp )
expr.display()

<IPython.core.display.Math object>

## Relation between spin flip and bcs

In [55]:
expr2 = Sp * Pdag_q * Sm
expr2.display()

<IPython.core.display.Math object>

In [56]:
expr2.simplify().display()

<IPython.core.display.Math object>

## Unitary group operators

In [57]:
e_ = names.e_
expr = dr.simplify(e_[p,q])
expr.display()

<IPython.core.display.Math object>

In [58]:
Jp = SU2LatticeDrudge.DEFAULT_RAISE

In [59]:
dr.sum(Jp[p]).display()

<IPython.core.display.Math object>

In [60]:
vec = dr.sum(Jp[p])

In [61]:
type(vec)

drudge.drudge.Tensor

## Scratch

In [71]:
expr = dr.sum((p,names.A), (q, names.A), KroneckerDelta(p, q) * Pdag_q * P_p)
expr.display()

<IPython.core.display.Math object>

In [29]:
import inspect

In [30]:
lines = inspect.getsource(dr.normal_order)

In [31]:
print(lines)

    def normal_order(self, terms: RDD, **kwargs):
        """Normal order the terms in the RDD."""

        if len(kwargs) > 0:
            raise ValueError('Invalid keyword arguments', kwargs)

        symms = self.symms
        swapper = self.swapper
        resolvers = self.resolvers

        init = terms.map(lambda term: _NOState(
            pivot=1, front=2, term=term.canon4normal(symms.value)
        ))

        res = nest_bind(init, lambda x: _sort_vec(
            x, swapper=swapper, resolvers=resolvers.value
        ), full_balance=self.full_balance)

        return res.map(lambda x: x.term)



In [32]:
bcs_vecs = (dr.Pdag, dr.N, dr.P)

In [33]:
expr = names.P_dag[p]

In [34]:
expr.base in bcs_vecs

True